In [1]:
# import pymysql
# from sqlalchemy import create_engine
# import sys
# from abc import ABC, abstractmethod
# from typing import *

import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 0. 모듈의 상위 패키지를 파이썬 경로로 설정

In [2]:
import sys

In [3]:
sys.path.append("C:\Pseudonymizer\pseudonymizer")
# /path/to/your/module

#### 1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드

In [4]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [5]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "0123")

In [6]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")

In [7]:
SQL = input("SQL 쿼리문 입력변수 = ")
    # SELECT * FROM DATA_FINANCE

In [8]:
queryObject.dataQueryLanguage(sql = SQL)

In [ ]:
# DATA_FINANCE_TUPLE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [10]:
DATA_FINANCE_TABLE = queryObject.executeQueryAsDataFrame()

In [11]:
type(DATA_FINANCE_TABLE)
DATA_FINANCE_TABLE.info()
DATA_FINANCE_TABLE.tail(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NUM_SERIAL                  95000 non-null  int64 
 1   NAME                        95000 non-null  object
 2   GENDER                      95000 non-null  object
 3   AGE                         95000 non-null  int64 
 4   PHONE_NUMBER                95000 non-null  object
 5   ZIP_CODE                    95000 non-null  int64 
 6   HOME_ADDRESS                95000 non-null  object
 7   HOME_TYPE                   95000 non-null  object
 8   INCOME_BRACKET              95000 non-null  int64 
 9   CREDIT_SCORE                95000 non-null  int64 
 10  REPAYMENT_RISK_INDEX        95000 non-null  int64 
 11  AMT_CREDITCARD_PAYMENT      95000 non-null  int64 
 12  AMT_CASHADVANCE_PAYMENT     95000 non-null  int64 
 13  NUM_CREDITCARD_ISSUANCES    95000 non-null  in

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,REPAYMENT_RISK_INDEX,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,26,8488084,7377056,8,2,N,0,0,N,0
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,7,5109440,3625570,3,2,N,0,0,N,0
94999,94999,안전웅,male,46,010-9622-1090,10585,경기도 고양시 덕양구 안진6길 12(지축동),오피스텔,7,44,76,874939,5785351,7,0,N,0,0,N,0


#### 2. 가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환

In [12]:
from pseudonymizer.pseudonymizer import Pseudonymizer

In [13]:
from pseudonymizer.pseudonym import Pseudonym
    # ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)

In [26]:
help(Pseudonym)

Help on class Pseudonym in module pseudonymizer.pseudonym:

class Pseudonym(builtins.object)
 |  Pseudonym(dataframe)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe)
 |      원본정보(재현데이터)와 가명처리 구체 클래스를 인스턴스 변수로 선언하는(초기화) 생성자
 |  
 |  __str__(self)
 |      캡슐화된 데이터셋의 속성(컬럼)정보를 반환하는 메서드
 |  
 |  addDictionary(self, column, pseudonymizers)
 |      가명처리를 수행할 데이터 컬럼명과 해당 열에 적용할 여러 가명처리 기법 리스트를 입력받아 다양한 비식별 조치를 수행할 수 있도록 지정하는 메서드
 |  
 |  addPseudonymizer(self, pseudonymizer)
 |      가명처리 추상 클래스에 대한 자식 클래스를 입력받는 pseudonymizer파라미터를 가지는 메서드
 |  
 |  categorizeEquivalentClass(self, attributes: List[str])
 |      각 행(레코드)에 대한 개인식별가능정보 속성(컬럼)들 사이에 동질 집합을 확인하는 메서드
 |      Pseudonym(dataframe).equivalent_class.keys()를 통해 동질집합 확인
 |  
 |  countEquivalentClass(self)
 |  
 |  getPseudonymizedDataframe(self)
 |      가명처리 데이터 반환
 |  
 |  pseudonymizeData(self)
 |      가명처리 기법을 해당 컬럼에 적용하는 메서드(apply함수를 활용하여 데이터프레임 모든 행, 특정 열에 비식별조치를 취하는 접근방식)
 |  
 |  -------------------------------------

In [14]:
# 가명처리 기법 적용 객체 생성
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE
)

In [16]:
# 원본 데이터의 컬럼 정보 출력(.info())
# print(pseudonym_object)

In [18]:
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object.categorizeEquivalentClass(
    attributes = ["NUM_CREDITCARD_ISSUANCES", "TF_LOAN", "TF_PENSION"]
)

In [21]:
# 동질집합 반환
pseudonym_object.countEquivalentClass()

(1, 'N', 'N') 7951
(1, 'N', 'Y') 596
(1, 'Y', 'N') 840
(1, 'Y', 'Y') 83
(2, 'N', 'N') 7988
(2, 'N', 'Y') 580
(2, 'Y', 'N') 883
(2, 'Y', 'Y') 54
(3, 'N', 'N') 8031
(3, 'N', 'Y') 585
(3, 'Y', 'N') 895
(3, 'Y', 'Y') 69
(4, 'N', 'N') 7960
(4, 'N', 'Y') 627
(4, 'Y', 'N') 902
(4, 'Y', 'Y') 68
(5, 'N', 'N') 7972
(5, 'N', 'Y') 573
(5, 'Y', 'N') 923
(5, 'Y', 'Y') 82
(6, 'N', 'N') 7857
(6, 'N', 'Y') 618
(6, 'Y', 'N') 868
(6, 'Y', 'Y') 71
(7, 'N', 'N') 7887
(7, 'N', 'Y') 593
(7, 'Y', 'N') 878
(7, 'Y', 'Y') 77
(8, 'N', 'N') 8002
(8, 'N', 'Y') 611
(8, 'Y', 'N') 803
(8, 'Y', 'Y') 64
(9, 'N', 'N') 7967
(9, 'N', 'Y') 598
(9, 'Y', 'N') 890
(9, 'Y', 'Y') 50
(10, 'N', 'N') 7899
(10, 'N', 'Y') 639
(10, 'Y', 'N') 906
(10, 'Y', 'Y') 60


In [30]:
# 가명처리 기법 클래스 입력
pseudonym_object.addPseudonymizer(
    pseudonymizer = None
)

입력받은 None 기술은 가명처리 기법에 추가할 수 없습니다.


In [32]:
# 적용 컬럼과 여러 가명처리 기법 정의
pseudonym_object.addDictionary(
    column = None,
    pseudonymizers = []
)

In [33]:
# 가명처리 수행
pseudonym_object.pseudonymizeData()

In [34]:
# 가명정보 반환
pseudonym_data = pseudonym_object.getPseudonymizedDataframe()

#### 3. 가명처리 절차 : 가명정보에 프라이버시 보호 모델 적용

In [25]:
from pseudonymizer.privacyPreservingModels import PrivacyPreservingModel

In [27]:
help(PrivacyPreservingModel)

Help on class PrivacyPreservingModel in module pseudonymizer.privacyPreservingModels:

class PrivacyPreservingModel(builtins.object)
 |  PrivacyPreservingModel(dataframe, epsilon)
 |  
 |  개인식별가능정보 속성을 기준으로 그룹화된 데이터로 프라이버시 보호 모델을 적용하여 정량적인 위험성을 규정하는 실행 클래스
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe, epsilon)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      동질집합에 대한 정보를 문자열로 반환하는 메서드
 |  
 |  applyDPrivacy(self, method, boundary, attributes, sensitive_attribute, outlier)
 |      차분 프라이버시 기법(차등적 정보보호 기능)을 적용하는 메서드
 |  
 |  applyKAnonymityOrLDiversity(self, method: str, K: int, L: int, attributes: List[str], sensitive_attribute)
 |      K-익명성과 L-다양성 모델을 선택적으로 적용하는 메서드
 |      input
 |      -----
 |      method: 프라이버시 보호 모델 메서드를 받고, 
 |      keyword arguments에 딕셔너리 형식으로 각 기법에 필요한 파라미터를 받아옴
 |  
 |  applyLocalLDiversity(self, K: int, L: int, attributes: List[str], sensitive_attribute: str, LocalL: int)
 |  
 |  apply